### QED-C Application-Oriented Benchmarks - Qiskit Version
The notebook contains a suite of application-oriented benchmarks for the Qiskit API.
Configure and run the cell below with the desired execution settings.
Then execute the remaining cells, each containing one benchmark program.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
min_qubits=2
max_qubits=10
skip_qubits=1
max_circuits=1
num_shots=1000

backend_id="qasm_simulator"
#backend_id="statevector_simulator"

hub="ibm-q"; group="open"; project="main"
provider_backend = None
exec_options = {}

# # ==========================
# # *** If using IBMQ hardware, run this once to authenticate
# from qiskit import IBMQ
# IBMQ.save_account('YOUR_API_TOKEN_HERE')

# # *** If you are part of an IBMQ group, set hub, group, and project name here
# hub="YOUR_HUB_NAME"; group="YOUR_GROUP_NAME"; project="YOUR_PROJECT_NAME"

# # *** This example shows how to specify an IBMQ backend using a known "backend_id"
# exec_options = { "optimization_level":3, "use_sessions":True, "resilience_level":1}
# backend_id="ibmq_belem"

# # ==========================
# # *** If using Azure Quantum, use this hub identifier and specify the desired backend_id
# # Identify your resources with env variables AZURE_QUANTUM_RESOURCE_ID and AZURE_QUANTUM_LOCATION
# hub="azure-quantum"; group="open"; project="QED-C App-Oriented Benchmarks - Qiskit Version"
# backend_id="<YOUR_BACKEND_NAME_HERE>"

# # ==========================
# The remaining examples illustrate other backend execution options

# # An example using IonQ provider
# from qiskit_ionq import IonQProvider
# provider = IonQProvider()   # Be sure to set the QISKIT_IONQ_API_TOKEN environment variable
# provider_backend = provider.get_backend("ionq_qpu")
# backend_id="ionq_qpu"

# # An example using BlueQubit provider (backend_id=CPU/QPU, device=cpu/qpu)
# import sys
# sys.path.insert(1, "../..")
# import os, bluequbit, _common.executors.bluequbit_executor as bluequbit_executor
# provider_backend = bluequbit.init()
# backend_id="BlueQubit-CPU"
# exec_options = { "executor": bluequbit_executor.run, "device":"cpu" }

# # An example using a typical custom provider backend (e.g. AQT simulator)
# import os
# from qiskit_aqt_provider import AQTProvider
# provider = AQTProvider(os.environ.get('AQT_ACCESS_KEY'))    # get your key from environment
# provider_backend = provider.backends.aqt_qasm_simulator_noise_1
# backend_id="aqt_qasm_simulator_noise_1"

# # Fire Opal can be used to manage executions on other backends, as illustrated here
# import _common.executors.fire_opal_executor as fire_opal_executor
# from _common.executors.fire_opal_executor import FireOpalBackend
# ibm_backend_id = "ibmq_jakarta"
# backend_id = f"fire_opal_{ibm_backend_id}"
# provider_backend = FireOpalBackend(ibm_backend_id=ibm_backend_id, hub=hub, group=group, project=project, token=token)
# exec_options = {"executor": fire_opal_executor.run}


In [3]:
# Custom optimization options can be specified in this cell (below is an example)

# # Example of pytket Transformer
# import _common.transformers.tket_optimiser as tket_optimiser
# exec_options.update({ "optimization_level": 0, "layout_method":'sabre', "routing_method":'sabre', "transformer": tket_optimiser.high_optimisation })

# # Define a custom noise model to be used during execution
# import _common.custom.custom_qiskit_noise_model as custom_qiskit_noise_model
# exec_options.update({ "noise_model": custom_qiskit_noise_model.my_noise_model() })

# # Example of mthree error mitigation
# import _common.postprocessors.mthree.mthree_em as mthree_em
# exec_options.update({ "postprocessor": mthree_em.get_mthree_handlers(backend_id, provider_backend) })


In [4]:
import hamlib_simulation_benchmark, hamlib_simulation_kernel, hamlib_utils
import execute
import metrics

execute.verbose = False
execute.verbose_time = False
hamlib_simulation_benchmark.verbose = False
hamlib_simulation_kernel.verbose = False
hamlib_utils.verbose = False

hamlib_simulation_kernel.global_U = None
hamlib_simulation_kernel.global_enc = None
hamlib_simulation_kernel.global_ratio = None
hamlib_simulation_kernel.global_rinst = None
hamlib_simulation_kernel.global_h = None
hamlib_simulation_kernel.global_pbc_val = None

### Hamiltonian Simulation - All Methods

#### Hamiltonian Simulation - tfim or hisenberg

In [ ]:
import sys
sys.path.insert(1, "hamlib/qiskit")
# exec_options = {"noise_model" : None}

# Set the hamiltonian_name: 'TFIM', 'Fermi-Hubbard-1D', 'Bose-Hubbard-1D', 'Heisenberg' or 'Max3Sat'
i = 1
for method in [2]:
    for hamiltonian_name in ['tfim']:
        for pbc_val in ['pbc','nonpbc']:
            for h in [4,6]:
                exec_options = {"noise_model" : None}
                hamlib_simulation_kernel.global_h = h
                hamlib_simulation_kernel.global_pbc_val = pbc_val
                print('Circuit Set', i, 'out of 4')
                print('Method, Hamiltonian, h, pbc_val = ',method, hamiltonian_name, h, pbc_val)
                print('=======================================================')
                metrics.data_suffix = f'_method_{method}_{hamiltonian_name}_{pbc_val}_h_{h}_noiseless'

                hamlib_simulation_benchmark.run(min_qubits=min_qubits, max_qubits=max_qubits, skip_qubits=skip_qubits,
                                max_circuits=max_circuits, num_shots=num_shots,
                                method=method, hamiltonian = hamiltonian_name, init_state=None,
                                backend_id=backend_id, provider_backend=provider_backend,
                                hub=hub, group=group, project=project, exec_options=exec_options
                                )
                i = i + 1

#### Hamiltonian Simulation - fermi-hubbard or bose-hubbard

In [ ]:
import sys
sys.path.insert(1, "hamlib/qiskit")
# exec_options = {"noise_model" : None}

# Set the hamiltonian_name: 'TFIM', 'Fermi-Hubbard-1D', 'Bose-Hubbard-1D', 'Heisenberg' or 'Max3Sat'
i = 1
for method in [2]:
    for hamiltonian_name in ['Fermi-Hubbard-1D']:
        for pbc_val in ['pbc','nonpbc']:
            for U in [8]:
                for enc in ['bk', 'jw', 'parity']:
                    exec_options = {"noise_model" : None}
                    hamlib_simulation_kernel.global_U = U
                    hamlib_simulation_kernel.global_enc = enc
                    hamlib_simulation_kernel.global_pbc_val = pbc_val
                    print('Circuit Set', i, 'out of 6')
                    print('Method, Hamiltonian, pbc_val, U, enc = ',method, hamiltonian_name, pbc_val, U, enc)
                    print('=======================================================')
                    metrics.data_suffix = f'_method_{method}_fh_{pbc_val}_U_{U}_enc_{enc}_noiseless'

                    hamlib_simulation_benchmark.run(min_qubits=min_qubits, max_qubits=max_qubits, skip_qubits=skip_qubits,
                                    max_circuits=max_circuits, num_shots=num_shots,
                                    method=method, hamiltonian = hamiltonian_name, init_state=None,
                                    backend_id=backend_id, provider_backend=provider_backend,
                                    hub=hub, group=group, project=project, exec_options=exec_options
                                    )
                    i = i + 1

#### Hamiltonian Simulation - Max3Sat

In [ ]:
import sys
sys.path.insert(1, "hamlib/qiskit")
# exec_options = {"noise_model" : None}

# Set the hamiltonian_name: 'TFIM', 'Fermi-Hubbard-1D', 'Bose-Hubbard-1D', 'Heisenberg' or 'Max3Sat'
i = 1
for method in [2]:
    for hamiltonian_name in ['Max3Sat']:
        for ratio in [2,3,4,5]:
            for rinst in ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09']:
                exec_options = {"noise_model" : None}
                hamlib_simulation_kernel.global_ratio = ratio
                hamlib_simulation_kernel.global_rinst = rinst
                print('Circuit Set', i, 'out of 40')
                print('Method, Hamiltonian, ration, rinst = ',method, hamiltonian_name, ratio, rinst)
                print('=======================================================')
                metrics.data_suffix = f'_method_{method}_max3sat_ratio_{ratio}_rinst_{rinst}_noiseless'

                hamlib_simulation_benchmark.run(min_qubits=min_qubits, max_qubits=max_qubits, skip_qubits=skip_qubits,
                                max_circuits=max_circuits, num_shots=num_shots,
                                method=method, hamiltonian = hamiltonian_name, init_state=None,
                                backend_id=backend_id, provider_backend=provider_backend,
                                hub=hub, group=group, project=project, exec_options=exec_options
                                )
                i = i + 1